In [1]:
import numpy as np
import pandas as pd
import math

# Define the user preference and destination feature vectors

# Naziv mesta,Cena smestaja za 1 osobu za 1 nocenje (€),Udaljenost od centra (m),Mogući broj putnika po sobi,Cena prevoza po osobi (€),Tip destinacije
# url = 'https://github.com/jovanrankovic02/AIS-Travel-Recommender-System/blob/main/TravelDestinations.csv?raw=true'
# data = pd.read_csv(url)

destination_pool = [
    ("Makarska", 40, 800, 3, 80, "Letovanje"),
    ("Sarajevo", 100, 1500, 2, 120, "Zimovanje"),
    ("Sveti Stefan", 15, 300, 6, 40, "Letovanje"),
    ("Miami", 60, 2000, 4, 150, "Letovanje"),
    ("Bangkok", 90, 2500, 3, 100, "Letovanje"),
    ("Sydney", 75, 3500, 2, 130, "Letovanje"),
    ("Los Angeles", 60, 4000, 5, 200, "Letovanje"),
    ("Dubai", 70, 5000, 4, 180, "Letovanje"),
    ("Tokyo", 33.33, 6000, 3, 100, "Letovanje"),
    ("Reykjavik", 110, 7000, 2, 150, "Zimovanje"),
    ("Amsterdam", 110, 8000, 1, 80, "Letovanje"),
    ("Cape Town", 76.67, 9000, 3, 140, "Letovanje"),
    ("Rio de Janeiro", 95, 10000, 2, 120, "Letovanje"),
    ("New York City", 42.5, 11000, 4, 160, "Letovanje"),
    ("Santorini", 66.67, 12000, 3, 140, "Letovanje"),
    ("Mauritius", 125, 13000, 2, 180, "Letovanje"),
    ("Maldives", 30, 14000, 5, 100, "Letovanje"),
    ("Bora Bora", 67.5, 15000, 4, 200, "Letovanje"),
    ("Barcelona", 60, 16000, 3, 120, "Letovanje"),
    ("Lisbon", 70, 17000, 2, 90, "Letovanje"),
    ("Prague", 160, 18000, 1, 100, "Letovanje"),
    ("Berlin", 40, 19000, 3, 80, "Zimovanje"),
    ("Paris", 45, 20000, 2, 90, "Letovanje"),
    ("Rome", 25, 21000, 4, 60, "Letovanje"),
    ("London", 43.33, 22000, 3, 80, "Letovanje"),
    ("Istanbul", 55, 23000, 2, 70, "Letovanje"),
    ("Singapore", 250, 24000, 1, 150, "Letovanje"),
    ("Seoul", 60, 25000, 3, 120, "Letovanje"),
    ("Honolulu", 150, 26000, 2, 200, "Letovanje"),
    ("Shanghai", 70, 27000, 4, 140, "Letovanje"),
    ("Moscow", 73.33, 28000, 3, 150, "Letovanje"),
    ("Buenos Aires", 42.5, 29000, 2, 110, "Letovanje"),
    ("Cancun", 26, 30000, 5, 130, "Letovanje"),
    ("Marrakech", 75, 31000, 4, 180, "Letovanje"),
    ("Kyoto", 76.67, 32000, 3, 150, "Letovanje"),
    ("Cairo", 105, 33000, 2, 120, "Letovanje"),
    ("Havana", 80, 34000, 1, 80, "Letovanje"),
    ("Vienna", 23.33, 35000, 3, 70, "Letovanje"),
    ("Vancouver", 150, 36000, 2, 200, "Zimovanje"),
    ("Toronto", 70, 37000, 4, 180, "Zimovanje"),
    ("Budapest", 43.33, 38000, 3, 160, "Letovanje"),
    ("Edinburgh", 70, 39000, 2, 140, "Letovanje"),
    ("Auckland", 24, 40000, 5, 120, "Letovanje"),
    ("Bariloche", 50, 41000, 4, 100, "Zimovanje"),
    ("Cusco", 60, 42000, 3, 80, "Zimovanje"),
    ("Prague", 160, 43000, 2, 70, "Letovanje"),
    ("Krakow", 60, 44000, 1, 60, "Letovanje"),
    ("Bruges", 50, 45000, 3, 50, "Letovanje")
]

In [2]:
import random

def generate_random_user_preferences():
    total_budget = random.randint(500, 10000)
    number_of_travelers = random.randint(1, 5)
    preferred_distance = random.randint(0, 50000)
    preferred_type = random.choice(["Letovanje", "Zimovanje"])
    
    return (total_budget, number_of_travelers, preferred_distance, preferred_type)

# Generisanje i ispis slučajnih korisničkih preferencija
random_preferences = generate_random_user_preferences()

In [3]:
# Define hyperparameters
num_clones = 48
num_iterations = 20
mutation_rate = 0.14
alpha_1, alpha_2, alpha_3 = 1, 1, 1

In [4]:
from statistics import mean
from statistics import stdev

def calculate_affinity(destination, user_preferences):
    destination_name, accommodation_price, distance, max_passengers, transportation_price, destination_type = destination
    total_budget, number_of_travelers, preferred_distance, preferred_type = user_preferences
    
    #normalizacija
    accommodation_price = (accommodation_price - mean(accommodation_price)) / stdev(accommodation_price)
    distance = (distance - mean(distance)) / stdev(distance)
    max_passengers = (max_passengers - mean(max_passengers)) / stdev(max_passengers)
    transportation_price = (transportation_price - mean(transportation_price)) / stdev(transportation_price)

    penalty = 0

    # Penalizacija ako se tip destinacije ne poklapa
    if destination_type != preferred_type:
        dest_penalty += 10000  # Velika kazna razmotri

    # Penalizacija ako je destinacija dalje od zeljenje udaljenosti
    distancePenalty = max(0, distance - preferred_distance)

    # Ukupni troskovi puta
    total_cost = (accommodation_price * number_of_travelers) + (transportation_price * number_of_travelers)
    
    # Penalizacija ako ukupni troškovi prelaze budžet
    costPenalty = max(0, total_cost - total_budget)

    # Izračunavanje afiniteta na osnovu udaljenosti od centra i troškova, uz penalizaciju
    affinity = alpha_1 * distancePenalty + alpha_2 * costPenalty + alpha_3 * dest_penalty
    return affinity

In [5]:
def generate_clones(selected_destinations, num_clones):
    # Generate clones for each selected destination
    clones = []
    for destination in selected_destinations:
        clones.extend([destination] * num_clones)
    return clones

In [6]:
def apply_mutation(clones, mutation_rate):
    # Mutate the clones with a certain mutation rate
    mutated_clones = []
    for clone in clones:
        if np.random.rand() < mutation_rate:
            # Apply some mutation, for simplicity we'll just alter the distance by a small random amount
            mutated_clone = list(clone)
            mutated_clone[2] = max(0, mutated_clone[2] + np.random.randint(-5, 5))  # Mutate the distance, ensure non-negative distance
            mutated_clones.append(tuple(mutated_clone))
        else:
            mutated_clones.append(clone)
    return mutated_clones

In [7]:
def recommend_travel_destination(user_preferences, destination_pool, num_clones, num_iterations, mutation_rate):
    memory_cells = []

    for iteration in range(num_iterations):
        # Calculate affinities
        affinities = [calculate_affinity(destination, user_preferences) for destination in destination_pool]

        # Select the best matching destinations
        selected_indices = np.argsort(affinities)[:num_clones]
        selected_destinations = [destination_pool[i] for i in selected_indices]

        # Generate clones
        clones = generate_clones(selected_destinations, num_clones)

        # Apply mutation
        mutated_clones = apply_mutation(clones, mutation_rate)

        # Evaluate the mutated clones
        clone_affinities = [calculate_affinity(clone, user_preferences) for clone in mutated_clones]

        # Select the best clone
        best_clone_index = np.argmin(clone_affinities)
        best_clone = mutated_clones[best_clone_index]

        # Update memory cells
        memory_cells.append(best_clone)

    # Select the best memory cell as the final recommendation
    memory_affinities = [calculate_affinity(cell, user_preferences) for cell in memory_cells]
    best_memory_index = np.argmin(memory_affinities)
    best_memory_cell = memory_cells[best_memory_index]

    return best_memory_cell

In [8]:
# Preporuči destinaciju
recommended_destination = recommend_travel_destination(random_preferences, destination_pool, num_clones, num_iterations, mutation_rate)
print(f"Preporučena destinacija: {recommended_destination}")

TypeError: 'int' object is not iterable